# TP6

### `Resolver usando Pandas`

Resolver los ejercicios del TP3 utilizando la librería Pandas.

### Ejercicio 1: Cargar Datos de ventas.

El archivo datos.dat contiene el registro de las ventas realizadas.

Tiene un formato de ancho fijo:
- `fecha`:    10 lugares
- `producto`: 30 lugares
- `precio`:   10 lugares
- `cantidad`:  5 lugares

Hacer una funcion que cargue los datos en un DataFrame de Pandas.

In [198]:
import pandas as pn # type: ignore
def cargar_datos():
    
    columnas = ['fecha', 'producto', 'precio', 'cantidad']
    colspecs = [(0, 10), (10, 40), (40, 50), (50, None)]  
    
    
    datos = pn.read_fwf("datos.dat", colspecs=colspecs, header=None, names=columnas)
    
    
    datos['fecha'] = datos['fecha'].str.strip()
    datos['producto'] = datos['producto'].str.strip()
    
    
    datos['precio'] = datos['precio'].astype(float)
    datos['cantidad'] = datos['cantidad'].astype(int)
    
    return datos
    

datos = cargar_datos()
print (datos)

         fecha    producto  precio  cantidad
0   2024-08-27     Mirinda  1510.0        14
1   2024-08-27     Mirinda  1560.0        12
2   2024-08-28     Torasso   940.0         8
3   2024-08-29  Pepsi Cola  1210.0        10
4   2024-08-30     Mirinda  1520.0         1
5   2024-09-01     Mirinda  1550.0        15
6   2024-09-01      Sprite   810.0         4
7   2024-09-02   Coca Cola  1100.0         4
8   2024-09-02  Pepsi Cola  1220.0        13
9   2024-09-02     Torasso   910.0         5
10  2024-09-02     Torasso   920.0         3
11  2024-09-03   Coca Cola  1020.0         8
12  2024-09-03     Mirinda  1570.0         7
13  2024-09-03     Mirinda  1590.0         2
14  2024-09-04  Pepsi Cola  1220.0        13
15  2024-09-05     Mirinda  1500.0         3
16  2024-09-05  Pepsi Cola  1300.0         5
17  2024-09-06   Coca Cola  1080.0         1
18  2024-09-06      Sprite   860.0        12
19  2024-09-06     Torasso   930.0         3
20  2024-09-07   Coca Cola  1080.0        14
21  2024-0

### Ejercicio 2: Calcular el total de ventas.
Hacer una función que sume los importes vendidos (precio * cantidad) y las cantidades.


In [199]:
def calcular_totales(datos):
    
    datos['importe'] = datos['precio'] * datos['cantidad']
    importetotal = datos['importe'].sum()
    cantidadtotal = datos['cantidad'].sum()
    
    return importetotal, cantidadtotal


importe, cantidad = calcular_totales(datos)

print(f"Las ventas fueron de ${importe:.2f} en {cantidad} unidades")

Las ventas fueron de $392730.00 en 335 unidades


### Ejercicio 3: Listar las unidades vendidas.
Listar cuántas unidades se vendieron en total para cada producto


In [200]:
def unidades_vendidas(datos):
    ventas_por_producto = datos.groupby('producto')['cantidad'].sum()
    return ventas_por_producto


ventas = unidades_vendidas(datos)
ventas_ordenadas = ventas.sort_index()


for producto, cantidad in ventas_ordenadas.items():
    print(f"{producto}: {cantidad} unidades vendidas")

Coca Cola: 57 unidades vendidas
Mirinda: 85 unidades vendidas
Pepsi Cola: 89 unidades vendidas
Sprite: 72 unidades vendidas
Torasso: 32 unidades vendidas


###  Ejercicio 4: Listar el precio promedio por producto.
Hacer un listado del precio promedio por producto.


In [201]:
def precio_promedio(datos):
    promedio_precio = datos.groupby('producto')['precio'].mean()
    return promedio_precio


promedios = precio_promedio(datos)

for producto, promedio in promedios.items():
    print(f"{producto}: precio promedio ${promedio:.2f}")

Coca Cola: precio promedio $1072.50
Mirinda: precio promedio $1545.83
Pepsi Cola: precio promedio $1245.00
Sprite: precio promedio $841.43
Torasso: precio promedio $920.00


### Ejercicio 5: Ranking de productos
Realizar un listado de los 3 productos más vendidos ordenados por la cantidad de unidades vendidas (ordenadas de mayor a menor)


In [202]:
def ranking_productos(datos, top=3):
    ventas_por_producto = unidades_vendidas(datos)
    ranking = ventas_por_producto.sort_values(ascending=False).head(top)
    
    return ranking


ranking = ranking_productos(datos)

print ("TOP 3 productos mas vendidos:")
for producto, cantidad in ranking.items():
    
    print(f"{producto}: {cantidad} unidades vendidas")




TOP 3 productos mas vendidos:
Pepsi Cola: 89 unidades vendidas
Mirinda: 85 unidades vendidas
Sprite: 72 unidades vendidas


### Ejercicio 6: Listar las ventas por mes
Realizar un listado del total de unidades vendidas por producto separado por mes.


In [203]:
def ventas_por_mes(datos):
   datos['mes'] = datos['fecha'].str[:7]
   ventas_mensuales = datos.groupby(['mes', 'producto'])['cantidad'].sum()
   return ventas_mensuales


ventas_por_mes(datos)

mes      producto  
2024-08  Mirinda       27
         Pepsi Cola    10
         Torasso        8
2024-09  Coca Cola     57
         Mirinda       58
         Pepsi Cola    79
         Sprite        72
         Torasso       24
Name: cantidad, dtype: int64

### Ejercicio 7: Informe general

Mostrar un listado de productos ordenados alfabeticamente que contengan el precio promedio, la cantidad de unidades vendidas y el importe total vendido para cada producto

In [204]:
def resumen_ventas(df):
    resumen = df.groupby('producto').agg(
        cantidad_total=('cantidad', 'sum'),
        precio_total=('precio', 'sum'),
        importe_total=('cantidad', lambda x: (df.loc[x.index, 'precio'] * x).sum())
    ).reset_index()

    resumen['precio_promedio'] = resumen['importe_total'] / resumen['cantidad_total']
    
    return resumen

def informe_ventas(resumen):
    for _, row in resumen.iterrows():
        print(f"{row['producto']}: Precio Promedio: ${row['precio_promedio']:.2f}, Unidades Vendidas: {row['cantidad_total']}, Importe Total: ${row['importe_total']:.2f}")

resumen = resumen_ventas(datos)
informe_ventas(resumen)


Coca Cola: Precio Promedio: $1066.32, Unidades Vendidas: 57, Importe Total: $60780.00
Mirinda: Precio Promedio: $1542.12, Unidades Vendidas: 85, Importe Total: $131080.00
Pepsi Cola: Precio Promedio: $1241.69, Unidades Vendidas: 89, Importe Total: $110510.00
Sprite: Precio Promedio: $847.78, Unidades Vendidas: 72, Importe Total: $61040.00
Torasso: Precio Promedio: $916.25, Unidades Vendidas: 32, Importe Total: $29320.00


## `Resolver usando NumPy`
## Resolver el ejercicio 2 del tp1 usando NumPy

### Ejercicio 8

Escribe una función en Python que encuentre los valores de `a`, `b`, y `c` para que la función cuadrática `f(x) = a x^2 + b x + c` pase exactamente por los siguientes puntos:

| x  | y  |
|---:|---:|
|  0 |  0 |
|  1 |  8 |
|  2 | 12 |
|  3 | 12 |
|  5 |  0 |

### Requisitos:
- La función debe explorar posibles valores de `a`, `b`, y `c` utilizando un método de prueba y error.
- Debe devolver los valores que hagan que la diferencia entre la función `f(x)` y los valores medidos `y` sea exactamente cero para cada punto.

> **Pista**: Los valores de `a`, `b`, y `c` son números pequeños.

La idea es implementar el mismo algoritmo que se uso en el TP1 pero usando NumPy en lugar de Python puro.

In [205]:
import numpy as np
def f(x, coeficientes):
    a,b,c = coeficientes
    return a*x**2 + b*x + c

def error(y, y_pred):
    return y - y_pred

X = np.array([0,1,2,3,5])
Y = np.array([0,8,12,12,0])

def buscar_coeficientes():
    coeficientes = np.polyfit(X, Y, 2)
    return coeficientes

coeficientes = buscar_coeficientes()
coeficientes

array([-2.00000000e+00,  1.00000000e+01, -1.27105749e-14])

### Ejercicio 9: Resolver el ejercicio 3 del TP1 usando NumPy
Buscar los coeficientes de la función que minimice la suma de los cuadrados de las diferencias entre los valores medidos y los valores de la función.

1. Crear un array con los coeficientes elegidos al azar (usar `randint(-10,10,3)`).
2. Calcular el valor de la función y el error correspondiente.
3. Mientras que el error sea mayor a 1:
    1. Definir nuevos coeficientes agregándoles un pequeño valor al azar a los coeficientes actuales (aprendizaje = 0.001).
    2. Si el error para los nuevos coeficientes es menor que el anterior, reemplazar los coeficientes actuales por los nuevos.


In [206]:
import numpy as np
from numpy.random import randint

def f(x, coeficientes):
    a, b, c = coeficientes
    return a*x**2 + b*x + c

def error(y, y_pred):
    return np.sum((y - y_pred)**2)

X = np.array([0, 1, 2, 3, 5])
Y = np.array([0, 8, 12, 11, 1])  

def buscar_coeficientes():
    
    coeficientes = randint(-10, 10, 3)
    
    
    y_pred = f(X, coeficientes)
    current_error = error(Y, y_pred)
    
    
    aprendizaje = 0.001

    while current_error > 1:
        
        nuevos_coeficientes = coeficientes + np.random.randn(3) * aprendizaje
        y_pred_nuevo = f(X, nuevos_coeficientes)
        new_error = error(Y, y_pred_nuevo)
        
        if new_error < current_error:
            coeficientes = nuevos_coeficientes
            current_error = new_error
            
    return coeficientes

# Ejecutamos la búsqueda de coeficientes
coeficientes = buscar_coeficientes()
print(f"Coeficientes encontrados: {coeficientes}")

Coeficientes encontrados: [-1.78045532  8.96289433  0.62688259]
